# Check Amat by Pauli basis

In the data folder, there are Amat in Pauli basis which we generated at 'RoM-handbook' repository.

Here, we check the Amat by this Pauli basis Amat.


In [1]:
import numpy as np
import scipy.sparse
from openfermion import QubitOperator, get_sparse_operator
from tqdm.auto import tqdm

In [2]:
def _pauli(pauli_id, qubit_pos) -> QubitOperator:
    if pauli_id == 0:
        return QubitOperator("")
    else:
        return QubitOperator("IXYZ"[pauli_id] + str(qubit_pos))


def pauli(pauli_id_list, qubit_pos_list) -> QubitOperator:
    if type(pauli_id_list) == int and type(qubit_pos_list) == int:
        return _pauli(pauli_id_list, qubit_pos_list)
    else:
        assert type(pauli_id_list) == list and type(qubit_pos_list) == list
        ret = QubitOperator("")
        for _pauli_id, _qubit_pos in zip(pauli_id_list, qubit_pos_list):
            ret = _pauli(_pauli_id, _qubit_pos) * ret
        return ret


def idx_to_pauli(i, n_qubit):
    quart_string = np.base_repr(i, 4).zfill(n_qubit)
    return pauli(list(map(int, quart_string)), list(range(n_qubit)))


def idx_to_pauli_mat(i, n_qubit):
    quart_string = np.base_repr(i, 4).zfill(n_qubit)
    pauli_id_list = list(map(int, quart_string))
    return get_sparse_operator(
        pauli(pauli_id_list, list(range(n_qubit))), n_qubits=n_qubit
    ).toarray()


def extract_pure_state(dm, shift=1e-10):
    values, vectors = np.linalg.eigh(dm)
    assert np.count_nonzero(np.abs(values) < shift) == len(values) - 1
    assert np.isclose(np.max(values), 1)
    ret = vectors[:, np.argmax(values)]
    first_nonzero_idx = np.argmax(np.abs(ret) > shift)
    if ret[first_nonzero_idx] < 0:
        ret *= -1
    return ret


def check_Amat(Amat: np.ndarray, Amat_in_Pauli: np.ndarray, pauli_matrices: np.ndarray):
    # Check if the made Amat is correct
    # All stabilizer states have 2^n Pauli matrices which stabilizes the state
    for idx, state in tqdm(enumerate(Amat.T), total=Amat.T.shape[0], desc="check_Amat"):
        for idx2 in range(Amat_in_Pauli.shape[0]):
            if Amat_in_Pauli[idx2, idx] == 0:
                continue
            pauli_matrix = pauli_matrices[idx2]
            sign = Amat_in_Pauli[idx2, idx]
            assert np.allclose(sign * pauli_matrix @ state, state)


def make_Amat(n: int, Amat_in_Pauli: np.ndarray, pauli_matrices: np.ndarray):
    # density matrix |psi><psi| = sum_i c_i |i><i| (by definition of c_i)
    # |psi> = extract_pure_state(|psi><psi|)
    _Amat = []
    for col in tqdm(Amat_in_Pauli.T, desc=f"make_Amat n={n}"):
        dm = sum(c * mat for c, mat in zip(col, pauli_matrices)) / 2**n
        _Amat.append(extract_pure_state(dm))
    Amat = np.array(_Amat).T
    assert np.allclose(np.linalg.norm(Amat, axis=0), 1)
    if n <= 3:
        check_Amat(Amat, Amat_in_Pauli, pauli_matrices)
    return Amat

In [3]:
from exputils.Amat import get_Amat


def post_process(Amat):
    eps = 1e-10
    first_nonzero_index = np.argmax(np.abs(Amat) > eps, axis=0)
    Amat /= Amat[first_nonzero_index, np.arange(Amat.shape[1])]
    Amat /= np.linalg.norm(Amat, axis=0)
    return np.array(
        sorted(
            Amat.T.tolist(),
            key=lambda x: (
                np.round(np.array(x).real, 5).tolist()
                + np.round(np.array(x).imag, 5).tolist()
            ),
        )
    ).T


for n in [1, 2, 3, 4]:
    pauli_matrices = np.array([idx_to_pauli_mat(i, n) for i in range(4**n)])
    Amat_in_Pauli = scipy.sparse.load_npz(
        f"../data/Amat_in_Pauli/Amat_in_Pauli{n}.npz"
    ).toarray()
    Amat_check = post_process(make_Amat(n, Amat_in_Pauli, pauli_matrices))
    Amat_data = post_process(get_Amat(n))
    assert np.allclose(Amat_check, Amat_data)

make_Amat n=1:   0%|          | 0/6 [00:00<?, ?it/s]

check_Amat:   0%|          | 0/6 [00:00<?, ?it/s]

make_Amat n=2:   0%|          | 0/60 [00:00<?, ?it/s]

check_Amat:   0%|          | 0/60 [00:00<?, ?it/s]

make_Amat n=3:   0%|          | 0/1080 [00:00<?, ?it/s]

check_Amat:   0%|          | 0/1080 [00:00<?, ?it/s]

make_Amat n=4:   0%|          | 0/36720 [00:00<?, ?it/s]

Now, we checked the validity of the Amat.
